# GPT4 with COT template

In [2]:
import chromadb
from chromadb.utils import embedding_functions

cot_template_chromadb_path = "../chromadb/"
client = chromadb.PersistentClient(path=cot_template_chromadb_path)

In [3]:
collection = client.get_collection(
    name="cot_template",
)

In [17]:
import openai
from dotenv import load_dotenv

load_dotenv(verbose=True)

openai_client = openai.OpenAI()

def chat_completions(messages):
    response = openai_client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        max_tokens=4096,
        temperature=0
    )
    return response

In [4]:
USER_PROMPT = """
{cot_template1}

{cot_template2}

{cot_template3}

## Question: {question}
{answer_choices}

## Answer
model generated chain of thought explanation
Therefore, the answer is [final model answer (e.g. 1,2,3,4,5)]
"""

def build_messages(question, answer_choices, cot_template1, cot_template2, cot_template3):
    return [
        {
            "role": "user",
            "content": USER_PROMPT.format(
                question=question,
                answer_choices=answer_choices,
                cot_template1=cot_template1,
                cot_template2=cot_template2,
                cot_template3=cot_template3
            )
        }

    ]

In [5]:
def answer_with_cot_template(item):
    embedding = openai_client.embeddings.create(input = [item["question"]], model="text-embedding-ada-002").data[0].embedding
    
    cot_template = collection.query(
        query_embeddings=[embedding],
        n_results=3,
    )

    response = chat_completions(
        messages=build_messages(
            question=item["question"],
            answer_choices=item["answer_choices"],
            cot_template1=cot_template['documents'][0][0],
            cot_template2=cot_template['documents'][0][1],
            cot_template3=cot_template['documents'][0][2],
        )
    )
    return response

In [10]:
import pandas as pd

exam74_df = pd.read_csv('../data/simple-gpt4/exam-74-result.csv')
exam74_false_df = exam74_df[exam74_df['result'] == False]


20

In [15]:
exam74_false_list = exam74_false_df.to_dict('records')

In [24]:
result74 = []
for i, item in enumerate(exam74_false_list):
    print(i)
    response = answer_with_cot_template(item)
    item['response_with_cot'] = response.choices[0].message.content
    result74.append(item)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [25]:
result74_df = pd.DataFrame(result74)
result74_df.to_csv('../data/gpt4-with-cot-template/exam-74-result.csv', index=False)

In [1]:
import pandas as pd

result74_manual_format_df = pd.read_csv('../data/gpt4-with-cot-template/exam-74-result-manual-format.csv')

In [3]:
print(len(result74_manual_format_df[result74_manual_format_df['result_with_cot'] == True]) / len(result74_manual_format_df) * 100)

45.0
